University of Michigan - ROB 101 Computational Linear Algebra

# Lab 5: Linear independence, Linear Combinations, and Existence and Uniqueness of Solutions to Ax=b

#### Purpose:  Learn to use Julia to better undersand linear independence and linear combinations
- Skills: 
    - More practice writing functions
    - LU vs LDLT
- Knowledge:
    - Working with Matrix Factorizations
    - Building confidence with complex functions

## Example 1: Finding the Number of Linearly Independent Vectors with LU and LDLT Methods

<img src = "https://i.postimg.cc/g2tJSmPY/ldlt.png" width = 700>

### LDLT Factorization vs LU Factorization

Why do we need to use "enhanced" LU factorization above? Let's explore a (not so uncommon) example of where LU breaks down for the purposes of counting the number of linearly independent vectors.

In [ ]:
# Run me, don't change me
using LinearAlgebra
n=4
A=zeros(n,n)
for i=1:n
    if i<n
        A[i,i+1]=1.0
    end
end
@show A

We see above that A has 3 independent columns, right? Well, let's try and factorize A using LU, and run our "foolproof" method to find how many linearly independent columns there are.

In [ ]:
# Run me, don't change me
# A is already upper triangular, but we run LU nevertheless
F=lu(A,check=false)
@show diag(F.U)
U=F.U

Oh no! The diagonal of U is all zeros, so we would be misled into thinking that A has no linearly independent columns, which we know is wrong. How can we fix this?

### The LDLT method 
Let's try to consider $A^\top \cdot A$ instead of just A.

In [ ]:
# Run me, don't change me
# A is already upper triangular, but we run LU nevertheless
F=lu(A'*A,check=false)
@show diag(F.U)
U=F.U

We now see that the diagonal of U is what we would expect it to be - three nonzero elements, indicating three independent columns. But wait, that was still run only using LU factorization! So what's the point of LDLT factorization?

### Let's hunt for a counter example

Below is our implementation of the LDLT algorithm:

In [ ]:
# Run me, don't change me!
function ldltROB101(A)
    epsilon = 1e-12
    M = A'*A
    n,m = size(A)
    Areduced = M
    L = Array{Float64,2}(undef,m,0)
    Id = zeros(m,m) + I
    P = Id    
    D=zeros(m,m)
    for i=1:m
        ii=argmax(diag(Areduced[i:m,i:m]))
        mrow=ii[1]+(i-1)
        if !(i==mrow)
            P[[i,mrow],:]=P[[mrow,i],:]
            Areduced[[i,mrow],:]=Areduced[[mrow,i],:]
            Areduced[:,[i,mrow]]=Areduced[:,[mrow,i]]
        end
        if (i>1)
            L[[i,mrow],:] = L[[mrow,i],:]
        end
        pivot=Areduced[i,i]
        if !isapprox(pivot,0,atol=epsilon)
            D[i,i]=pivot
            C=Areduced[:,i]/pivot
            L=[L C]
            Areduced=Areduced-(C*pivot*C')
        else
            L=[L Id[:,i:m]]
            break
        end
    end
    diagD=diag(D)
    return L,P,D,diagD
end

We'll run a loop until we find an example where LU--> diag(U) and LDLT --> diag(D) give different "predictions" for the number of linearly independent columns of A.

*For the record, LDLT is always correct. We are doing this to show that if you try (incorrectly) to use LU for determining the number of linearly independent columns of a matrix, it will sometimes mess up. Hence, we really did need to learn LDLT*

In [ ]:
# Run me don't change me
using Random
function CounterExample()
    Random.seed!(4356)
    flag = 1
    N=5
    n=floor(Int,N/2)
    k=0
    while flag > 0
        k=k+1
        # Build a matrix A
        B=randn(N,N-1)
        C=randn(n,n)
        A=[B[:,1:n] B[:,1:n]*C B[:,n+n:end]]
        # Apply LU to A'A
        F=lu(A'*A, check=false)
        diagU=diag(F.U)
        indicesLU=findall(x->x<1e-10, abs.(diagU))
        NumLinIndepLU=N-length(indicesLU)
        # Apply LDLT to A'A
        L,P,D,diagD = ldltROB101(A)
        indicesLDLT=findall(x->x<1e-10, diagD)
        NumLinIndepLDLT=N-length(indicesLDLT)
        # Check for a discrepancy in their reported number of linearly indep columns
        if (NumLinIndepLDLT > NumLinIndepLU)||(k>1e5)
            return k, A, F, L, P, D, diagD
            flag=0
        end        
    end
end

function cleanUp(A,tol=1e-10)
    # Zero out small entries of a matrix or vector
    B=copy(A)
    indicesSmall=findall(x->x<tol, abs.(B))
    B[indicesSmall]=0.0*B[indicesSmall]
return B
end

In [ ]:
k, A, F, L, P, D, diagD = CounterExample()

In [ ]:
# The diagonoal of F.U, showing which elements are nonzero
# run me don't change me
cleanUp(diag(F.U))


In [ ]:
# The diagonoal of diagD, showing which elements are nonzero
# run me don't change me
cleanUp(diagD)

In [ ]:
# run me don't change me

# Gives the LDLT factorized upper triangular matrix

cleanUp(D*L')

In [ ]:
# Run me don't change me

# Multiplying by transpose(P) on the right of A puts
#   the independent columns to the front of the matrix

# This is very convenient
barA=A*P'

We see that the first two columns of A are linearly independent, but the first three columns are not:

In [ ]:
# Run me don't change me

newA01=A[:,1:3]
L,P,D,diagD=ldltROB101(newA01)
diagD

We see now that the first three columns of A\*P' are linearly independent

In [ ]:
# Run me don't change me

newA02=barA[:,1:3]
L,P,D,diagD=ldltROB101(newA02)
diagD

### Bottom line, we can use the LDLT factorization applied to $A^\top \cdot A$ to determine the number of linearly independent columns in A and we form $\bar{A} = A \cdot P^\top$ to move the independent columns to the front of the matrix.

## Problem 1. Number of Linearly Independent Vectors

Using the provided ```ldltROB101``` function, and the above method of determing the number of linearly independent vectors in a matrix, complete the function ```num_independent_vectors``` which takes in a matrix and returns the number of linearly independent vectors.

In [ ]:
function num_independent_vectors(A)
    # Your code here!
    
    
end

In [ ]:
# public autograder cell, yay!

A = [1 2 3; 4 5 6; 7 8 10]
a = num_independent_vectors(A)
@show a
T1 = @assert a == 3

B = [1 2 3; 4 5 6; 2 4 6]
b = num_independent_vectors(B)
@show b
T2 = @assert b == 2

@show [T1 T2]

## Problem 2. Testing for linear combinations

<img src = "https://i.postimg.cc/2j9KStm1/lin-com-testing.png" width = 700>

You will now use the function you just wrote in order to determine whether a vector is a linear combination of a matrix. Complete the function ```is_linear_combo``` with takes in a vector v and a matrix A and returns true if v is a linear combination of A and false otherwise

In [ ]:
function is_linear_combo(A, v)
    
end

In [ ]:
# public autograder cell, yay!

A = [3.5 1.0 5.0; 5.0 2.0 6.0; 6.5 3.0 7.0; 8.0 4.0 8.0]
v = [4; 4; 4; 4]
T1 = @assert is_linear_combo(A, v)

A = [3.5 1.0 5.0; 5.0 2.0 6.0; 6.5 3.0 7.0; 8.0 4.0 8.0]
v = [20; 11; 12; 14]
T2 = @assert !is_linear_combo(A, v)

@show [T1 T2]

## Problem 3. Putting it all Together

<img src = "https://i.postimg.cc/MKPM4y72/existence-and-uniqueness.png" width = 700>

You have written a function to find out how many linearly independent vectors are in a matrix. You have written a second function to determine is a vector is a linear combination of a matrix, or set of vectors. Now, finally, you will utilize these two functions to complete the ```exists_and_unique``` function below, which takes in a matrix A and vector b, and returns whether the system Ax = b has a unique solution.

In [ ]:
function exists_and_unique(A,b)
    
end

In [ ]:
# public autograder cell, yay!

A = [1 2 3; 4 5 6; 7 8 10]
v = [12; 15; 19]
T1 = @assert exists_and_unique(A,v)

@show [T1]

# Test some other matrices!

